In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
day_price_0_df = pd.read_csv("./round-3-island-data-bottle/prices_round_3_day_0.csv", delimiter=";")
day_price_1_df = pd.read_csv("./round-3-island-data-bottle/prices_round_3_day_1.csv", delimiter=";")
day_price_2_df = pd.read_csv("./round-3-island-data-bottle/prices_round_3_day_2.csv", delimiter=";")

# merge all the dataframes
merged_df = pd.concat([day_price_0_df, day_price_1_df, day_price_2_df])
merged_df["continuous_timestamp"] = merged_df["day"] * 1e6 + merged_df["timestamp"]
product_names = [
    "RAINFOREST_RESIN", "KELP", "SQUID_INK",
    "CROISSANTS", "JAMS", "DJEMBES",
    "PICNIC_BASKET1", "PICNIC_BASKET2",
    "VOLCANIC_ROCK",
    "VOLCANIC_ROCK_VOUCHER_10500",
    "VOLCANIC_ROCK_VOUCHER_10250",
    "VOLCANIC_ROCK_VOUCHER_10000",
    "VOLCANIC_ROCK_VOUCHER_9750",
    "VOLCANIC_ROCK_VOUCHER_9500",
]

products = {
    name: merged_df[merged_df["product"] == name].reset_index(drop=True)
    for name in product_names
}